In [24]:
import pandas as pd

train = pd.read_csv("train.csv")


print(train[0:10])

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   
5   6          50       RL         85.0    14115   Pave   NaN      IR1   
6   7          20       RL         75.0    10084   Pave   NaN      Reg   
7   8          60       RL          NaN    10382   Pave   NaN      IR1   
8   9          50       RM         51.0     6120   Pave   NaN      Reg   
9  10         190       RL         50.0     7420   Pave   NaN      Reg   

  LandContour Utilities  ... PoolArea PoolQC  Fence MiscFeature MiscVal  \
0         Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
1         Lvl    AllPub  ...       

In [25]:
types = train.dtypes


In [26]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

ke = train.columns[2]
print(train[ke])



0       RL
1       RL
2       RL
3       RL
4       RL
        ..
1455    RL
1456    RL
1457    RL
1458    RL
1459    RL
Name: MSZoning, Length: 1460, dtype: object


In [27]:
cat_dict = {}

cat_labels = []
new_arr = []
first = True
non_cat = []

for ke, type in zip(train.columns, types):
    if type == "object":
        le = OneHotEncoder()
        if train[ke].isnull().values.any():
            ifnan = True
        else:
            ifnan = False

        if ifnan:
            unique = np.expand_dims(np.asarray(["nan"] + list(train[ke].dropna().unique())), axis = 1)
        else:
            unique = np.expand_dims(np.asarray(list(train[ke].dropna().unique())), axis = 1)


        train[ke] = train[ke].fillna("nan")

        cat = le.fit_transform(np.expand_dims(train[ke], axis = 1)).toarray().T
        if first:
            first = False
            categorical= cat
        else:
            categorical = np.concatenate((categorical, cat), axis = 0)

        keys = le.categories_
        ka = [k for k in keys]
        cat_dict[ke] = ka
        cat_labels.extend(ka)
    else:
        non_cat.append(ke)

    


In [28]:
categorical = categorical.T



In [29]:
train_no_lab = train.iloc[:,1:-1]
train_lab = train.iloc[:,-1]

train_non_cat = train_no_lab[[t for t in train_no_lab.columns if t in non_cat]]

train_arr = np.concatenate((train_non_cat, categorical), axis = 1)

print(train_arr.shape)





(1460, 304)


In [30]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 5)

imputer.fit(train_arr)
train_arr = imputer.transform(train_arr)
print(train_arr.shape)

(1460, 304)


In [31]:
##### Try Linear Model
from sklearn.linear_model import LinearRegression
reg = LinearRegression(fit_intercept = False)
reg.fit(train_arr, train_lab)

LinearRegression(fit_intercept=False)

In [32]:
train_pred = reg.predict(train_arr)
train_score = reg.score(train_arr, train_lab)
print(train_score)

0.9333483603038338


In [36]:
#### Trying LASSO
from sklearn.linear_model import Lasso
